In [1]:
import numpy as np
from matplotlib import pyplot as plt
import os
from keras.api.models import Model
from keras.api.layers import Input, Dense, Activation, Flatten, LeakyReLU, BatchNormalization
from keras.api.optimizers import Adam
from keras.api.losses import binary_crossentropy
import random
import cv2
from sklearn.model_selection import train_test_split

In [2]:
work_dir = 'c:/Users/Jalil/Desktop/Ecole/M2TSI/tp2/'

In [3]:
dataset_toystory_path = os.path.join(work_dir, 'datasets/Movies/toy_story_1_01')
dataset_ernest_path = os.path.join(work_dir, 'datasets/Movies/ernest_celestine_01')
datasetnpy_path = os.path.join(work_dir, 'datasets/dataset.npy')

In [4]:
dataset = np.empty(shape=(1000, 540, 920, 3), dtype=np.float32)

In [6]:

i = 0
for image in os.listdir(dataset_toystory_path):
    dataset[i] = cv2.imread(os.path.join(dataset_toystory_path, image)) / 255.0
    i = i + 1

for image in os.listdir(dataset_ernest_path):
    dataset[i] = cv2.imread(os.path.join(dataset_ernest_path, image)) / 255.0
    i = i + 1
    
    

In [13]:
with open(datasetnpy_path, 'wb') as f:
    np.save(f, dataset)

In [14]:
with open(datasetnpy_path, 'rb') as f:
    dataset = np.load(f)

In [15]:
print(f'dataset_shape : {dataset.shape}')

dataset_shape : (1000, 540, 920, 3)


In [16]:
target_vector = np.zeros(1000)
target_vector[500:] = 1
target_vector = target_vector.reshape((-1,1))

In [17]:
print(f'target_shape : {target_vector.shape}')

target_shape : (1000, 1)


In [10]:
def compute_mean(image):
    image_size = image.shape[0] * image.shape[1] * image.shape[2]
    return [np.mean(image), np.mean(image[:,:,0]), np.mean(image[:,:,1]), np.mean(image[:,:,2])]

def extract_histogram(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    hist, bin = np.histogram(img_gray, bins=10)
    bin = bin[1:]
    return list((hist*bin/np.sum(hist)))

def extract_frequencies(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fft_image = np.log(np.abs(np.fft.fftshift(np.fft.fft2(gray_image))))
    
    hist, bin = np.histogram(fft_shift, bins=10)
    bin = bin[1:]

    return list((hist*bin)/np.sum(hist))
        
        

In [21]:
a = np.array([1,2,3])
b = np.array([4,5])
c = np.array([6,7])

print(np.concatenate([a, b, c]))

[1 2 3 4 5 6 7]


In [46]:
image_test = dataset[10]
image_gray = cv2.cvtColor(image_test, cv2.COLOR_RGB2GRAY)
print(np.array([np.mean(image_test), np.mean(image_test[:,:,0]), np.mean(image_test[:,:,1]), np.mean(image_test[:,:,2])]))

hist, bin = np.histogram(image_gray, bins=10)
bin = bin[1:]
print((hist*bin/np.sum(hist)))

fft_image = np.log(np.abs(np.fft.fftshift(np.fft.fft2(image_gray))))
hist1, bin1 = np.histogram(fft_image, bins=10)
bin1 = bin1[1:]
print((hist1*bin1)/np.sum(hist1))

features = np.concatenate([
    np.array([np.mean(image_test), np.mean(image_test[:,:,0]), np.mean(image_test[:,:,1]), np.mean(image_test[:,:,2])]),
    (hist*bin/np.sum(hist)),
    (hist1*bin1)/np.sum(hist1)
])

print('------------------------------')
print(features)

[0.39232388 0.24620844 0.39247617 0.53828686]
[0.00445833 0.01488688 0.04759239 0.15103865 0.12113023 0.01943358
 0.02086896 0.01540419 0.01462319 0.00477858]
[-2.97986499e-04 -3.68434334e-03  1.13739702e-02  8.01749789e-01
  1.40593810e+00  5.70433224e-01  8.98949542e-02  9.77511793e-03
  4.60815228e-04  2.43741610e-05]
------------------------------
[ 3.92323881e-01  2.46208444e-01  3.92476171e-01  5.38286865e-01
  4.45833340e-03  1.48868762e-02  4.75923932e-02  1.51038650e-01
  1.21130233e-01  1.94335757e-02  2.08689610e-02  1.54041870e-02
  1.46231880e-02  4.77858293e-03 -2.97986499e-04 -3.68434334e-03
  1.13739702e-02  8.01749789e-01  1.40593810e+00  5.70433224e-01
  8.98949542e-02  9.77511793e-03  4.60815228e-04  2.43741610e-05]


In [47]:
def extract_features(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    
    hist, bin = np.histogram(image_gray, bins=10)
    bin = bin[1:]
    
    fft_image = np.log(np.abs(np.fft.fftshift(np.fft.fft2(image_gray))))
    hist1, bin1 = np.histogram(fft_image, bins=10)
    bin1 = bin1[1:]

    return np.concatenate([
        np.array([np.mean(image_test), np.mean(image_test[:,:,0]), np.mean(image_test[:,:,1]), np.mean(image_test[:,:,2])]),
        (hist*bin/np.sum(hist)),
        (hist1*bin1)/np.sum(hist1)
    ])

In [48]:
print(extract_features(dataset[10]))

[ 3.92323881e-01  2.46208444e-01  3.92476171e-01  5.38286865e-01
  4.45833340e-03  1.48868762e-02  4.75923932e-02  1.51038650e-01
  1.21130233e-01  1.94335757e-02  2.08689610e-02  1.54041870e-02
  1.46231880e-02  4.77858293e-03 -2.97986499e-04 -3.68434334e-03
  1.13739702e-02  8.01749789e-01  1.40593810e+00  5.70433224e-01
  8.98949542e-02  9.77511793e-03  4.60815228e-04  2.43741610e-05]


In [12]:
features_tab = np.empty(shape=(dataset.shape[0], 24))
for image in dataset:
    features_tab.append(extract_features(image))
features_tab = np.array(features_tab)

In [13]:
print(features_tab.shape)
print(features_tab[0])

(1000, 24)
[ 3.42181772e-01  3.99085462e-01  3.43309939e-01  2.84150124e-01
  8.00046821e-04  4.99966712e-03  2.25589006e-02  3.47162649e-02
  2.46622774e-02  2.86294268e-02  6.35984971e-02  1.12241946e-01
  6.46359705e-02  6.70065571e-03 -1.64895523e-04 -5.19261152e-03
 -3.67121914e-02  8.71644639e-01  7.59822927e-01  2.47945940e-02
  2.04529753e-03  5.46325918e-04  8.27578604e-05  2.43062453e-05]


In [14]:
input = Input(shape=(24,))
x = Dense(units=32)(input)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dense(units=16)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
output = Dense(units=1, activation='sigmoid')(x)

model = Model(input, output)

In [15]:
model.summary()
optimizer = Adam(learning_rate=0.005)
model.compile(optimizer=optimizer, loss=binary_crossentropy, metrics=['accuracy'])

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,537 (6.00 KB)

 Trainable params: 1,441 (5.63 KB)

 Non-trainable params: 96 (384.00 B)

In [16]:
target_vector = target_vector.reshape((-1,1,1))
print(target_vector.shape)
print(features_tab.shape)

(1000, 1, 1)
(1000, 24)


In [17]:
x_train, x_test, y_train, y_test = train_test_split(features_tab, target_vector, train_size=0.8, shuffle=True)

In [18]:
x_train
print(f'x_train_shape : {x_train.shape}, y_train_shape : {y_train.shape}')

print(f'x_test_shape : {x_test.shape}, y_test_shape : {y_test.shape}')

x_train_shape : (800, 24), y_train_shape : (800, 1, 1)
x_test_shape : (200, 24), y_test_shape : (200, 1, 1)


model.fit(dataset, y_arr, epochs=5, batch_size=16, shuffle=True)

In [20]:
model.fit(x_train, y_train, batch_size=16, epochs=50, shuffle=True)

Epoch 1/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7133 - loss: 0.5821
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9136 - loss: 0.2456
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 0.9161 - loss: 0.2086
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step - accuracy: 0.9449 - loss: 0.1678
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9374 - loss: 0.1491
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9272 - loss: 0.1730
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9684 - loss: 0.0879
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9381 - loss: 0.1775 
Epoch 9/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9628 - loss: 0.1069 
Epoch 10/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9437 - loss: 0.1576
Epoch 11/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step - accuracy: 0.9450 - loss: 0.1283
Epoch 12/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9

In [21]:
model.evaluate(x_test, y_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9769 - loss: 0.0686  


[0.07598099857568741, 0.9700000286102295]

In [ ]:
image_test_path = r'C:\Users\Jalil\Desktop\Ecole\M2TSI\tp2\datasets\image_testjpg'

plt.imshow(image_test_path)